In [277]:
import pandas as pd
import numpy as np

In [278]:
movies_rating = pd.read_csv("https://datasets.imdbws.com/title.ratings.tsv.gz")
movies_rating_clean = movies_rating["tconst\taverageRating\tnumVotes"].str.split("\\t", expand=True)
movies_rating_clean.columns = ['ID', 'Note_moyenne', 'Nombre_de_votes']
movies_rating_clean['Note_moyenne'] = movies_rating_clean['Note_moyenne'].astype(float)
movies_rating_clean['Nombre_de_votes'] = movies_rating_clean['Nombre_de_votes'].astype(float)
movies_rating_filtré = movies_rating_clean[movies_rating_clean.Nombre_de_votes > 1999]

In [279]:
df = pd.read_csv('https://raw.githubusercontent.com/Jeremstar/Succes_de_films-IMDb/main/Database/IMDB_2000votes.csv', 
                 converters={"actor": lambda x: x.strip("[]").split(", "),
                             'director': lambda y : y.strip("[]").split(", "),
                            'genre': lambda z : z.strip("[]").split(", "),
                            'creator': lambda u : u.strip("[]").split(", ")}) 

movies_rating_filtré['url']='/title/'+movies_rating_filtré['ID']+'/'
df_fusionné = df.merge(movies_rating_filtré, on='url',how='left')

In [280]:
df_fusionné = df_fusionné.reindex(columns=['ID_y','name','alternateName','url','contentRating',
                                           'datePublished','genre','actor','director','creator','Origine','Budget',
                                           'duration','keywords','Note_moyenne','Nombre_de_votes','ID_x'])
#On drop creator, pour les questions de scrap évoquées plus tôt, et budget parce qu'il est impossible de l'obtenir avec
#uniquement 6 arguments de path comme le permet la library de scrap
df_fusionné =df_fusionné.drop(['ID_x', 'Budget', 'alternateName'],axis=1) 
df_fusionné.rename(columns={'ID_y':'ID'}, inplace= True)

In [281]:
def correcteur(colonne, old, new) :
    if type(df_fusionné[colonne][0]) == list :
        all_crews = []
        for crew in df_fusionné[colonne]:
            crew_corrigé = []
            if crew != [] :
                for individu in crew :
                    crew_corrigé.append(individu.replace(old, new))
            all_crews.append(crew_corrigé)
        df_fusionné[colonne] = all_crews
    else :
        df_fusionné[colonne]= df_fusionné[colonne].str.replace(old, new)

In [282]:
correcteur('actor', '&apos;', "'")
correcteur('director', '&apos;', "'")
correcteur('name', '&apos;', "'")

In [283]:
from pandas.core.common import flatten

def valeurs_possibles (colonne) :
    list_nonflat = df_fusionné[colonne]
    flat_list = list(flatten(list_nonflat))
    liste_valeurs = list(set(flat_list))
    return liste_valeurs

In [284]:
correcteur('contentRating', 'Tous Public', 'tous publics')
correcteur('contentRating', 'Tous Publics', 'tous publics')
correcteur('contentRating', 'Tous public', 'tous publics')
correcteur('contentRating', 'Tous publics', 'tous publics')
correcteur('contentRating', 'Not Rated', 'Not rated') #Absence de certificat
correcteur('contentRating', 'Unrated', 'Not rated') #Idem
correcteur('contentRating', '-12', '12')
correcteur('contentRating', '10 avec avertissement', '10')
correcteur('contentRating', '12 avec avertissement', '12')
correcteur('contentRating', 'Passed', 'Approved') #Classification d'avant 1968
correcteur('contentRating', '14+', '14')
correcteur('contentRating', '(Banned)', 'Banned')
correcteur('contentRating', '-16', '16')
correcteur('contentRating', 'TV-14', '14')
correcteur('contentRating', '16 avec avertissement', '16')
correcteur('contentRating', '-10', '10')
correcteur('contentRating', 'TV-PG', 'Accord parental')
correcteur('contentRating', 'PG-13', 'Accord parental')
correcteur('contentRating', 'M/PG', 'Accord parental')
correcteur('contentRating', 'PG', 'Accord parental')
correcteur('contentRating', '0+', 'tous publics')
correcteur('contentRating', 'E1Tous Publics+', 'tous publics')
correcteur('contentRating', 'E1Tous Public+', 'tous publics')
correcteur('contentRating', 'TV-13', '13')
correcteur('contentRating', 'R', '18')
correcteur('contentRating', 'GP', 'Accord parental')
correcteur('contentRating', 'MA-17', '17')
correcteur('contentRating', 'X', '18')
correcteur('contentRating', 'TV-Y7-FV', '7')
correcteur('contentRating', 'TV-Y7', '7')
correcteur('contentRating', 'TV-G', 'Tous Public')
correcteur('contentRating', 'G', 'Tous Public')
correcteur('contentRating', 'Tous Publics', 'tous publics')
correcteur('contentRating', 'Tout public', 'tous publics')
correcteur('contentRating', '1Tous Public', 'tous publics')
correcteur('contentRating', 'Tous Public+', 'tous publics')
correcteur('contentRating', 'NC-17', '18')
correcteur('contentRating', 'TV-MA', '18')
correcteur('contentRating', 'TV-T', 'tous publics')
correcteur('contentRating', 'M', '18')
correcteur('contentRating', 'E', 'Éducatif')
correcteur('contentRating', 'T', 'tous publics')
correcteur('contentRating', 'K-A', 'Erreur')
correcteur('contentRating', 'AO', 'Erreur')
correcteur('contentRating', 'Open', 'Erreur')
correcteur('contentRating', 'tous publicsV-Y', 'tous publics')
correcteur('contentRating', 'Éducatif1tous publics+', 'tous publics')
correcteur('contentRating', 'tous publicss avec avertissement', 'tous publics')
correcteur('contentRating', '1tous publics', 'tous publics')
correcteur('contentRating', 'tous publicss', 'tous publics')

In [285]:
df_fusionné['month'] = pd.DatetimeIndex(df_fusionné['datePublished']).month

In [286]:
def recodage_duree(duree) :
    if type(duree) == str :
        duree = duree[2:]
        if 'H' not in duree :
            duree_corrigee = int(duree[-3:-1])
        elif 'M' not in duree :
            duree_corrigee = int(duree[-2:-1])*60
        else :
            heures = int(duree[0])*60
            duree = duree[2:]
            minutes = int(duree[-3:-1])
            duree_corrigee = heures + minutes
    else :
        duree_corrigee = None
    return duree_corrigee

In [287]:
df_fusionné['duration']=df_fusionné['duration'].apply(lambda row : recodage_duree(row))

In [288]:
index_with_nan = df_fusionné.index[df_fusionné.loc[:,'datePublished'].isnull()]
df_fusionné.drop(index_with_nan,0, inplace=True)
df_fusionné = df_fusionné.sort_values(by='datePublished')
df_fusionné = df_fusionné.reset_index(drop = True)

In [290]:
import datetime
df_fusionné['datePublished'] = df_fusionné['datePublished'].apply(lambda x : datetime.datetime.strptime(str(x), '%Y-%m-%d'))

In [300]:
df_fusionné = df_fusionné.sort_values(by='datePublished')
#On initialise le dictonnaire à utiliser
actor_scores = {}
#On parcourt le df ; on considère la liste des acteurs et la note moyenne
for _, row in df_fusionné.iterrows():
    actors = row['actor']
    #On parcourt la liste des acteurs
    for i, actor in enumerate(actors):
        if actor not in actor_scores:
            # Si l'acteur n'a joué dans aucun film précédemment, son score initial est de 5
            actor_scores[actor] = []
        #On crée le score. Il est calculé comme : la somme du score à la période précédente, et de la note
        #du dernier film, multipliée par i+1 (de sorte à donner plus de poids aux films récents).
        #On divise ensuite par i+2, à la manière d'une moyenne pondérée. Les valeurs du score sont donc
        #comprises entre 1 et 10, et en général entre 5 et 10 sauf film catastrophique
        new_score = (actor_scores[actor][-1] + rating * (i + 1)) / (i + 2) if actor_scores[actor] else 5
        actor_scores[actor].append(new_score)
    rating = row['Note_moyenne']

In [302]:
%%capture 
#permet de ne pas afficher l'output
actor_scores.pop('')

In [303]:
df_fusionné['liste_scores'] = None
#Cette fonction permet de remplacer les listes vides par un None. Elle peut servir aussi pour le director
def replace_empty_list(x):
    if x == ['']:
        return None
    return x
#On remplace les listes d'acteur vides par des None
df_fusionné['actor'] = df_fusionné['actor'].apply(replace_empty_list)

In [304]:
for index, row in df_fusionné.iterrows():
    #print(index) - cette commande permet de se rassurer sur l'exécution du code, mais en vrai
    #il est suffisamment rapide pour ne pas paniquer
    liste_scores = []
    if type(row['actor']) == list : #on parcourt la liste des acteurs s'il y en a une
        for acteur in row['actor']: #pour chaque acteur du cast
            liste_scores.append(actor_scores[acteur][0]) #on ajoute le premier score du dico à la liste
            actor_scores[acteur].pop(0) #et on efface cette valeur:donc on peut toujours boucler sur la valeur 1
        df_fusionné.at[index, 'liste_scores'] = liste_scores #on intègre la liste obtenue
    else :
        df_fusionné.at[index, 'liste_scores'] = None #si on n'a pas d'acteur, on n'a pas de liste de score

In [305]:
def moyenne_pondérée(liste):
    if type(liste) == list :
        resultat = 0
        for i, score in enumerate(liste):
            resultat += score / (i + 2)
    else :
        resultat = None
    return resultat

df_fusionné['score_acteur'] = None
df_fusionné['score_acteur'] = df_fusionné['liste_scores'].apply(moyenne_pondérée)

In [306]:
director_scores = {}
for _, row in df_fusionné.iterrows():
    directors = row['director']
    for i, director in enumerate(directors):
        if director not in director_scores:
            director_scores[director] = []
        new_score = (director_scores[director][-1]+rating*(i+1))/(i+2) if director_scores[director] else 5
        director_scores[director].append(new_score)
    rating = row['Note_moyenne']

In [307]:
%%capture
director_scores.pop('')

In [308]:
df_fusionné['liste_scores2'] = None
df_fusionné['director'] = df_fusionné['director'].apply(replace_empty_list)

In [309]:
for index, row in df_fusionné.iterrows():
    liste_scores2 = []
    if type(row['director']) == list :
        for directeur in row['director']:
            if directeur != None :
                liste_scores2.append(director_scores[directeur][0])
                director_scores[directeur].pop(0)
        df_fusionné.at[index, 'liste_scores2'] = liste_scores2
    else :
        df_fusionné.at[index, 'liste_scores2'] = None

In [310]:
df_fusionné['score_director'] = None
df_fusionné['score_director'] = df_fusionné['liste_scores2'].apply(moyenne_pondérée)

In [311]:
creator_scores = {}
for _, row in df_fusionné.iterrows():
    creator = row['creator']
    for i, creator in enumerate(creator):
        if creator not in creator_scores:
            creator_scores[creator] = []
        new_score = (creator_scores[creator][-1]+rating*(i+1))/(i+2) if creator_scores[creator] else 5
        creator_scores[creator].append(new_score)
    rating = row['Note_moyenne']

In [312]:
%%capture
creator_scores.pop('')

In [313]:
df_fusionné['liste_scores3'] = None
df_fusionné['creator'] = df_fusionné['creator'].apply(replace_empty_list)

In [314]:
for index, row in df_fusionné.iterrows():
    liste_scores3 = []
    if type(row['creator']) == list :
        for createur in row['creator']:
            if createur != None :
                liste_scores3.append(creator_scores[createur][0])
                creator_scores[createur].pop(0)
        df_fusionné.at[index, 'liste_scores3'] = liste_scores3
    else :
        df_fusionné.at[index, 'liste_scores3'] = None

In [315]:
df_fusionné['score_creator'] = None
df_fusionné['score_creator'] = df_fusionné['liste_scores3'].apply(moyenne_pondérée)

In [316]:
#index_with_nan = df.index[df_fusionné.loc[:,'datePublished'].isnull()]
#df_fusionné.drop(index_with_nan,0, inplace=True)
import calendar
df_fusionné['month'] = pd.DatetimeIndex(df_fusionné['datePublished']).month
df_fusionné['month'] = df_fusionné['month'].astype(int)
df_fusionné['month_name'] = df_fusionné['month'].apply(lambda x: calendar.month_abbr[x])

In [318]:
df_fusionné

,ID,name,url,contentRating,datePublished,genre,actor,director,creator,Origine,...,Note_moyenne,Nombre_de_votes,month,liste_scores,score_acteur,liste_scores2,score_director,liste_scores3,score_creator,month_name
0,tt2221420,Sallie Gardner at a Gallop,/title/tt2221420/,tous publics,1878-06-15,"['Documentary', 'Short']","['Gilbert Domm', 'Sallie Gardner']",['Eadweard Muybridge'],['/company/co0405154/'],United States,...,7.40,3101.00,6,"[5, 5]",4.17,[5],2.50,[5],2.50,Jun
1,tt0392728,Roundhay Garden Scene,/title/tt0392728/,Not rated,1888-10-14,"['Documentary', 'Short']","['Annie Hartley', 'Adolphe Le Prince', 'Joseph...",['Louis Aimé Augustin Le Prince'],['/company/co0293112/'],United Kingdom,...,7.30,6257.00,10,"[5, 5, 5]",5.42,[5],2.50,[5],2.50,Oct
2,tt0000005,Blacksmith Scene,/title/tt0000005/,Not rated,1893-05-09,"['Short', 'Comedy']","['Charles Kayser', 'John Ott']",['William K.L. Dickson'],['/company/co0037035/'],United States,...,6.20,2557.00,5,"[5, 5]",4.17,[5],2.50,[5],2.50,May
3,tt0000008,Edison Kinetoscopic Record of a Sneeze,/title/tt0000008/,NaN,1894-01-09,"['Documentary', 'Short']",['Fred Ott'],['William K.L. Dickson'],['/company/co0037035/'],United States,...,5.40,2074.00,1,[5],2.50,[5.6],2.80,[5.6],2.80,Jan
4,tt0177707,Dickson Experimental Sound Film,/title/tt0177707/,Not rated,1894-08-31,"['Short', 'Music']",['William K.L. Dickson'],['William K.L. Dickson'],['/company/co0037035/'],United States,...,6.70,2379.00,8,[5],2.50,[5.5],2.75,[5.5],2.75,Aug
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
46076,tt14208870,The Fabelmans,/title/tt14208870/,Accord parental,2023-01-25,['Drama'],"['Michelle Williams', 'Gabriel LaBelle', 'Paul...",['Steven Spielberg'],"['/company/co0009119/', '/company/co0567243/',...",France,...,7.90,12305.00,1,"[6.725890245478375, 6.3999999999999995, 6.8580...",7.21,[6.9063875853549686],3.45,"[6.752108761737512, 6.888715277777777, 7.02509...",9.86,Jan
46077,tt14444726,Tár,/title/tt14444726/,18,2023-02-22,"['Drama', 'Music']","['Cate Blanchett', 'Noémie Merlant', 'Nina Hoss']",['Todd Field'],"['/company/co0042399/', '/company/co0081915/',...",France,...,7.80,12003.00,2,"[7.948827078334977, 7.33125, 7.537673611111112]",8.30,[6.975],3.49,"[7.542958868415903, 7.458333333333333, 7.50416...",9.66,Feb
46078,tt13833688,The Whale,/title/tt13833688/,18,2023-03-08,['Drama'],"['Brendan Fraser', 'Sadie Sink', 'Ty Simpkins']",['Darren Aronofsky'],"['/company/co0390816/', '/company/co0062935/',...",France,...,8.30,5448.00,3,"[7.596014441012787, 7.1499999999999995, 7.25]",7.99,[7.324999999999999],3.66,"[7.993516271615544, 7.650851980452675, 5]",7.80,Mar
46079,tt4960748,Till,/title/tt4960748/,Accord parental,2023-03-22,"['Biography', 'Drama', 'History']","['Danielle Deadwyler', 'Jalyn Hall', 'Frankie ...",['Chinonye Chukwu'],"['/company/co0054040/', '/company/co0028970/',...",France,...,7.20,4080.00,3,"[6.65, 5, 8.133076018467547]",7.02,[6.65],3.33,"[7.242351205914764, 5, 8.364698360973854, 5, 5...",10.33,Mar


In [141]:
def binariseur(df, nom_de_colonne):

    df = df.copy()# Créer une copie du dataframe

    if nom_de_colonne == 'genre' or nom_de_colonne == 'keywords': #pour les listes

      df_explosé_temp = df.explode (nom_de_colonne)

      Liste_elem_unique = df_explosé_temp[nom_de_colonne].unique() # je récupère la liste des éléments unique dans la colonne

      for element in Liste_elem_unique:

        df[element] = df[nom_de_colonne].apply(lambda x: 1 if element in x else 0) # nouvelle colonne avec un un ou un zéro

    else : 

      Liste_elem_unique = df[nom_de_colonne].unique() #si c'est pas des listes

    for element in Liste_elem_unique:

        df[element] = df[nom_de_colonne].apply(lambda x: 1 if element == x else 0)

        # Pour chaque élément unique,  je crée une nouvelle colonne dans la dataframe

    

    return df              

In [142]:
df_fusionné = binariseur(df_fusionné, 'genre')
df_fusionné = binariseur(df_fusionné, 'contentRating')
df_fusionné = binariseur(df_fusionné, 'month_name')

In [147]:
df_regression = df_fusionné.copy()

In [66]:
from sklearn import datasets
from sklearn.linear_model import Lasso
from sklearn.model_selection import train_test_split

In [72]:
from sklearn.linear_model import LinearRegression, Ridge, Lasso, LogisticRegression
from sklearn.ensemble import RandomForestRegressor


# Création de modèles de régression linéaire et de forêt aléatoire

models = [LinearRegression(), RandomForestRegressor(), Ridge(), Lasso()] #LogisticRegression()


from sklearn.metrics import mean_squared_error

from sklearn.metrics import r2_score

X = df_regression[['ID', 'duration', 'score_acteur', 'score_director', 'score_creator',

                   'Jun', 'Oct', 'May', 'Jan', 'Aug', 'Mar', 'Apr', 'Dec', 'Sep','Jul', 'Feb', 'Nov',

                    "'Short'", "'Comedy'", "'Documentary'", "'Horror'", "'Fantasy'",

                    "'Action'", "'Adventure'", "'Family'", "'Animation'", "'Crime'",

                    "'Drama'", "'History'", "'War'", "'Sport'", "'Romance'",

                    "'Mystery'", "'Thriller'", "'Sci-Fi'", "'Western'", "'Biography'",

                    "'Music'", "'Musical'", "'Film-Noir'", "'News'", "'Talk-Show'",

                    "'Adult'", "'Game-Show'", "'Reality-TV'",

                    'tous publics+','6+','9+', '16','17', '14', '18','Public Averti','7',

                    'Accord parental','(Banned)',

                    'Éducatif', 'Approved', 'Not rated', 'tous publics', '13', '12']].dropna()

y = df_regression[['ID', 'Note_moyenne']].dropna()

df_filtrage = pd.merge(X, y, on='ID')



X = df_filtrage[['duration', 'score_acteur', 'score_director', 'score_creator',

                   'Jun', 'Oct', 'May', 'Jan', 'Aug', 'Mar', 'Apr', 'Dec', 'Sep','Jul', 'Feb', 'Nov',

                    "'Short'", "'Comedy'", "'Documentary'", "'Horror'", "'Fantasy'",

                    "'Action'", "'Adventure'", "'Family'", "'Animation'", "'Crime'",

                    "'Drama'", "'History'", "'War'", "'Sport'", "'Romance'",

                    "'Mystery'", "'Thriller'", "'Sci-Fi'", "'Western'", "'Biography'",

                    "'Music'", "'Musical'", "'Film-Noir'", "'News'", "'Talk-Show'",

                    "'Adult'", "'Game-Show'", "'Reality-TV'",

                    'tous publics+','6+','9+', '16','17', '14', '18','Public Averti','7',

                    'Accord parental','(Banned)',

                    'Éducatif', 'Approved', 'Not rated', 'tous publics', '13', '12']]

y = df_filtrage['Note_moyenne']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)



# Liste pour stocker les résultats de chaque modèle

results = []



# Boucle sur chaque modèle

for model in models:

    # Entraînement du modèle sur les données d'entraînement

    model.fit(X_train,y_train)


    # Prédiction sur les données de test

    y_pred = model.predict(X_test)

    

    # Calcul de l'erreur quadratique moyenne sur les données de test

    mse = mean_squared_error(y_test, y_pred)

    

    #Calcul du R2

    r2 = r2_score(y_test, y_pred)



    # Ajout des résultats dans la liste

    results.append({'Model': model, 'MSE': mse, 'R2': r2})



# Tri de la liste par ordre croissant de MSE (et décroissant de R2)

results_triés_R2 = sorted(results, key=lambda x: x['R2'])



# Affichage des résultats

for result in results_triés_R2 : 

  print(f'Model: {result["Model"]}, MSE: {result["MSE"]}, R2: {result["R2"]}')


Model: Lasso(), MSE: 1.4702592555271818, R2: 0.14837881109027806
Model: LinearRegression(), MSE: 1.3519044290055107, R2: 0.21693371234097525
Model: Ridge(), MSE: 1.351675683210084, R2: 0.21706620922241093
Model: RandomForestRegressor(), MSE: 0.9584060585318556, R2: 0.4448605550641339


In [ ]:
#ridge => peu de corrélation entre nos variables, donc logique qu'il ne soit pas plus performant
#random forest performant => réussit à gommer l'overfitting, ce qui peut toucher nos données en revanche

In [236]:
#création d'un df de films inventés avec leurs caractéristiques pour obtenir une prédiction des notes
#en donnant des scores arbitraires, sans utiliser les fonctions score sur des acteurs/réalisateurs/creators
df_test = pd.DataFrame({'contentRating': ['tous publics'], 'genre':['Comedy'],
                        'duration': '138', 'score_acteur' : '8.5', 'score_director' : '9', 
                        'score_creator' : '6.2', 'month_name': 'Dec'}, index = [1])
#on prend uniquement les colonnes qui vont servir par la suite

In [195]:
df_test

,contentRating,genre,duration,score_acteur,score_director,score_creator,month_name
1,tous publics,Comedy,138,8.5,9,6.2,Dec


In [237]:
#le binariseur originel prend sa liste de valeurs uniques dans l'ensemble des valeurs des colonnes du df donné en argument, ici je mets plutot la fonction
#valeurs_possibles pour qu'il y ait toutes les valeurs même avec un df de 1 row en argument
#on enlève les nan au passage dans ces listes
import math

def binariseur(df, nom_de_colonne):

    df = df.copy()# Créer une copie du dataframe

    if nom_de_colonne == 'genre' or nom_de_colonne == 'keywords': #pour les listes

      df_explosé_temp = df.explode (nom_de_colonne)

      Liste_elem_unique = [x for x in valeurs_possibles(nom_de_colonne) if pd.isnull(x) == False] 

      for element in Liste_elem_unique:

        df[element] = df[nom_de_colonne].apply(lambda x: 1 if element in x else 0) # nouvelle colonne avec un un ou un zéro

    else : 

      Liste_elem_unique = [x for x in valeurs_possibles(nom_de_colonne) if pd.isnull(x) == False] #si c'est pas des listes

    for element in Liste_elem_unique:

        df[element] = df[nom_de_colonne].apply(lambda x: 1 if element == x else 0)

        # Pour chaque élément unique,  je crée une nouvelle colonne dans la dataframe

    

    return df 

In [238]:
df_test = binariseur(df_test,'genre')
df_test = binariseur(df_test, 'contentRating')
df_test = binariseur(df_test, 'month_name')

In [233]:
df_test

,contentRating,genre,duration,score_acteur,score_director,score_creator,month_name,,'Adventure','Drama',...,Nov,Jun,Oct,Mar,Apr,Feb,Jul,May,Dec,Sep
1,tous publics,Comedy,138,8.5,9,6.2,Dec,0,0,0,...,0,0,0,0,0,0,0,0,1,0


In [239]:
df_test.drop(['genre', 'contentRating','month_name','14+','Erreur',''], axis = 1, inplace = True)
#on enlève les noms de colonne qui, d'après les erreurs obtenues en lançant la prédiction, matchent pas avec les données d'entrée

In [158]:
#on teste mtn le modèle randomforest sur ce df

from sklearn.ensemble import RandomForestRegressor

model = RandomForestRegressor()

X = df_regression[['ID', 'duration', 'score_acteur', 'score_director', 'score_creator',

                   'Jun', 'Oct', 'May', 'Jan', 'Aug', 'Mar', 'Apr', 'Dec', 'Sep','Jul', 'Feb', 'Nov',

                    "'Short'", "'Comedy'", "'Documentary'", "'Horror'", "'Fantasy'",

                    "'Action'", "'Adventure'", "'Family'", "'Animation'", "'Crime'",

                    "'Drama'", "'History'", "'War'", "'Sport'", "'Romance'",

                    "'Mystery'", "'Thriller'", "'Sci-Fi'", "'Western'", "'Biography'",

                    "'Music'", "'Musical'", "'Film-Noir'", "'News'", "'Talk-Show'",

                    "'Adult'", "'Game-Show'", "'Reality-TV'",

                    'tous publics+','6+','9+', '16','17', '14', '18','Public Averti','7',

                    'Accord parental','(Banned)',

                    'Éducatif', 'Approved', 'Not rated', 'tous publics', '13', '12']].dropna()

y = df_regression[['ID', 'Note_moyenne']].dropna()

df_filtrage = pd.merge(X, y, on='ID')



X = df_filtrage[['duration', 'score_acteur', 'score_director', 'score_creator',

                   'Jun', 'Oct', 'May', 'Jan', 'Aug', 'Mar', 'Apr', 'Dec', 'Sep','Jul', 'Feb', 'Nov',

                    "'Short'", "'Comedy'", "'Documentary'", "'Horror'", "'Fantasy'",

                    "'Action'", "'Adventure'", "'Family'", "'Animation'", "'Crime'",

                    "'Drama'", "'History'", "'War'", "'Sport'", "'Romance'",

                    "'Mystery'", "'Thriller'", "'Sci-Fi'", "'Western'", "'Biography'",

                    "'Music'", "'Musical'", "'Film-Noir'", "'News'", "'Talk-Show'",

                    "'Adult'", "'Game-Show'", "'Reality-TV'",

                    'tous publics+','6+','9+', '16','17', '14', '18','Public Averti','7',

                    'Accord parental','(Banned)',

                    'Éducatif', 'Approved', 'Not rated', 'tous publics', '13', '12']]

y = df_filtrage['Note_moyenne']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

model.fit(X_train,y_train)

RandomForestRegressor()

In [241]:
df_test = df_test.reindex(columns=['duration', 'score_acteur', 'score_director', 'score_creator',

                   'Jun', 'Oct', 'May', 'Jan', 'Aug', 'Mar', 'Apr', 'Dec', 'Sep','Jul', 'Feb', 'Nov',

                    "'Short'", "'Comedy'", "'Documentary'", "'Horror'", "'Fantasy'",

                    "'Action'", "'Adventure'", "'Family'", "'Animation'", "'Crime'",

                    "'Drama'", "'History'", "'War'", "'Sport'", "'Romance'",

                    "'Mystery'", "'Thriller'", "'Sci-Fi'", "'Western'", "'Biography'",

                    "'Music'", "'Musical'", "'Film-Noir'", "'News'", "'Talk-Show'",

                    "'Adult'", "'Game-Show'", "'Reality-TV'",

                    'tous publics+','6+','9+', '16','17', '14', '18','Public Averti','7',

                    'Accord parental','(Banned)',

                    'Éducatif', 'Approved', 'Not rated', 'tous publics', '13', '12'])

In [242]:
y_pred = model.predict(df_test)

In [243]:
print(y_pred[0])

6.759000000000001
